In [2]:
import rasterio
from rasterio.mask import mask
from scipy.ndimage import generic_filter
import geopandas as gpd
import numpy as np
from pyproj import Proj, transform
import pandas as pd

In [6]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

portugal = world[world['name'] == 'Portugal']['geometry'].values[0]

print(portugal)

/tmp/ipykernel_3674/3069684263.py:1: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


POLYGON ((-9.034817674180246 41.880570583659676, -8.67194576662672 42.13468943945496, -8.263856980817792 42.28046865495034, -8.013174607769912 41.790886135417125, -7.422512986673795 41.79207469335984, -7.251308966490824 41.91834605566505, -6.6686055159676565 41.883386949219584, -6.389087693700915 41.381815497394655, -6.851126674822552 41.11108266861753, -6.864019944679385 40.33087189387483, -7.026413133156595 40.184524237624245, -7.066591559263529 39.711891587882775, -7.498632371439726 39.62957103124181, -7.098036668313128 39.03007274022379, -7.374092169616318 38.37305858006492, -7.029281175148796 38.07576406508977, -7.166507941099865 37.803894354802225, -7.537105475281024 37.42890432387624, -7.453725551778092 37.09778758396607, -7.855613165711986 36.83826854099627, -8.382816127953689 36.97888011326246, -8.898856980820327 36.86880931248078, -8.746101446965554 37.65134552667661, -8.83999752443988 38.266243394517616, -9.287463751655224 38.3584858261586, -9.526570603869715 38.737429104154

In [7]:
with rasterio.open('md75_grd/md75_grd/w001001.adf') as src:
    # Clip the raster to the extent of Portugal
    out_image, out_transform = mask(src, [portugal.__geo_interface__], crop=True)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

# Save the clipped raster
with rasterio.open('clipped.tif', 'w', **out_meta) as dest:
    dest.write(out_image)

# Now you can calculate the TRI on the clipped raster
with rasterio.open('clipped.tif') as src:
    # Read the raster data
    raster_data = src.read(1)

    def calculate_tri(dem):
        def tri_filter(x):
            central_value = x[len(x) // 2]
            return np.mean(np.abs(x - central_value))

        return generic_filter(dem, tri_filter, size=3)

    # Calculate TRI
    tri = calculate_tri(raster_data)


NaturalFires = pd.read_csv('NaturalFires.csv')

tri_values = []
for i in range(len(NaturalFires)):
    latitude = NaturalFires['latitude'][i]
    longitude = NaturalFires['longitude'][i]

    # Define the source CRS (WGS84)
    src_crs = Proj(init='EPSG:4326')

    # Get the CRS of the raster data
    dst_crs = Proj(src.crs)

    # Convert the coordinates to the raster data's CRS
    longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)

    # Convert the projected coordinates to raster indices
    row, col = src.index(longitude_proj, latitude_proj)

    # Check if the indices are within the dimensions of the array
    if 0 <= row < tri.shape[0] and 0 <= col < tri.shape[1]:
        tri_value = tri[row, col]
        print(tri_value)
        tri_values.append(tri_value)
    else:
        print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")
        tri_values.append(None)

NaturalFires['tri'] = tri_values

/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

17
7
37
15
35
34
37
18
12
8
9
8
3
7
20
11
9
17
8
11
46
45
17
55
14
15
23
14
10
29
17
17
7
15


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

17
28
0
10
12
15
11
22
13
22
16
27
22
11
24
27
17
17
7
14
18
9
17
3
6
3
3
6
12
11
13
5


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

...

/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

8
5
26
27
6
9
26
7
16
22
5
29
19
41
8
29
21
21
10
15
17
13
34
16
4
18
11
9
4
37
12
16
24
20


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

40
12
28
15
19
37
32
8
11
60


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/1446947568.py:44: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

In [9]:
from osgeo import gdal
import rasterio
from pyproj import Proj, transform

# Open the DEM file
with rasterio.open('clipped.tif') as src:
    # Calculate the slope
    gdal.DEMProcessing('slope.tif', src.name, 'slope')

# Open the slope file
with rasterio.open('slope.tif') as slope_ds:
    # Read the slope data
    slope_data = slope_ds.read(1)

slope_values = []
for i in range(len(NaturalFires)):
    latitude = NaturalFires['latitude'][i]
    longitude = NaturalFires['longitude'][i]

    # Define the source CRS (WGS84)
    src_crs = Proj(init='EPSG:4326')

    # Get the CRS of the raster data
    dst_crs = Proj(slope_ds.crs)

    # Convert the coordinates to the raster data's CRS
    longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)

    # Convert the projected coordinates to raster indices
    row, col = slope_ds.index(longitude, latitude)

    # Check if the indices are within the dimensions of the array
    if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
        slope_value = slope_data[row, col]
        print(slope_value)
        slope_values.append(slope_value)
    else:
        print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")
        slope_values.append(None)

NaturalFires['slope'] = slope_values

/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+ini

89.994255
89.98668
89.99731
89.99309
89.992744
89.99696
89.99686
89.99516
89.99359
89.98692
89.990204
89.98104
89.97518
89.98774
89.98998
89.99219
89.98938
89.99292
89.99038
89.99298
89.99721
89.991905
89.98866
89.99857
89.99404
89.98914
89.99647
89.99404
89.98726
89.9973
89.99519
89.99519


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

89.98866
89.98914
89.99332
89.99701
-9999.0
89.99054
89.993614
89.993546
89.992065
89.99486
89.99411
89.99486
89.99464
89.98933
89.995865
89.95852
89.99656
89.996185
89.99558
89.994934
89.98585
89.99385
89.9936
89.99187
89.994934
89.976036
89.98313
89.836235
89.97211
89.9849
89.97891
89.99004
89.99359
89.98145
89.86758
89.99096
89.98757
89.99096
89.99351
89.98964
89.99068


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

...

89.98752
89.988914
89.99683
89.99645
89.99843
89.97627
89.997086
89.980705
89.99735
89.997765
89.98794
89.99662
89.994865
89.99856
89.987526
89.99548
89.98792
89.9952
89.99631
89.994316
89.99767
89.99621
89.99756
89.99625
89.99078
89.996025
89.98927
89.97814
89.996124
89.99703
89.955086


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

89.991684
89.99691
89.988396
89.99506
89.99618
89.965225
89.99718
89.995834
89.997734
89.99008
89.99718
89.995224
89.99535
89.98873
89.995
89.995316
89.99252
89.99729
89.98973
89.98207
89.9945
89.99313
89.99068
89.97192
89.99756
89.99377
89.99502
89.99657
89.99625
89.99798


/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_3674/4116646575.py:27: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)
/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_strin

89.99188
89.99721
89.99339
89.99356
89.99782
89.99612
89.990265
89.99291
89.99861


In [18]:
from osgeo import gdal
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Path to the directory containing the .adf files
dem_directory = 'md75_grd/md75_grd'  # replace with your path

# Define the target coordinate system (EPSG:3857 is a common choice for meters)
dst_crs = 'EPSG:3857'

# Open the DEM file
with rasterio.open(dem_directory) as src:
    # Calculate the transform and the dimensions of the reprojected dataset
    transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    # Update the metadata
    kwargs = src.meta.copy()
    kwargs.update({
        'driver': 'GTiff',  # specify the driver to be used
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    # Reproject the dataset
    with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)

# Calculate the slope
gdal.DEMProcessing('slope.tif', 'reprojected.tif', 'slope')

# Open the slope file
with rasterio.open('slope.tif') as slope_ds:
    # Read the slope data
    slope_data = slope_ds.read(1)

# Define your coordinate
longitude = -8.1007731  # replace with your longitude
latitude = 41.3096543  # replace with your latitude

# Convert the coordinates to raster indices
row, col = slope_ds.index(longitude, latitude)

# Extract the slope value at these indices
slope_value = slope_data[row, col]

print(f"The slope at the coordinate ({longitude}, {latitude}) is {slope_value}.")

RasterioIOError: reprojected.tif: Free disk space available is 8093208576 bytes, whereas 104500576596 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.

In [ ]:
NaturalFires.to_csv('NaturalFiresTOPOGRAPHY.csv', index=False)

In [3]:
NaturalFires = pd.read_csv('NaturalFires.csv')

In [20]:
from osgeo import gdal

# Open the raster file
ds = gdal.Open('clipped.tif')  # replace 'clipped.tif' with your raster file

# Get the geotransform
gt = ds.GetGeoTransform()

# Get the raster dimensions
cols = ds.RasterXSize
rows = ds.RasterYSize

# Calculate the boundary coordinates
min_x = gt[0]
min_y = gt[3] + rows*gt[5]
max_x = gt[0] + cols*gt[1]
max_y = gt[3]

print(f"Raster extent: Min X: {min_x}, Min Y: {min_y}, Max X: {max_x}, Max Y: {max_y}")

# Close the raster file
ds = None

Raster extent: Min X: -179.99999999999991, Min Y: -89.99589934699992, Max X: 180.00000000000009, Max Y: 83.63410065300008


In [4]:
from osgeo import gdal

# Open the DEM file
dem = gdal.Open('clipped.tif')  # replace 'clipped.tif' with your DEM file

# Calculate the slope
gdal.DEMProcessing('slope.tif', dem, 'slope')

# Close the DEM file
dem = None

# Open the slope file
slope_ds = gdal.Open('slope.tif')

slopes = []
for i in range(len(NaturalFires)):
    latitude = NaturalFires['latitude'][i]
    longitude = NaturalFires['longitude'][i]

    # Convert the geographic coordinates to raster indices
    gt = slope_ds.GetGeoTransform()
    col = int((longitude - gt[0]) / gt[1])
    row = int((latitude - gt[3]) / gt[5])

    # Read the slope data
    slope_data = slope_ds.ReadAsArray()

    # Check if the indices are within the dimensions of the array
    if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
        slope_value = slope_data[row, col]
        print(slope_value)
        slopes.append(slope_value)
    else:
        print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")
        slopes.append(None)

# Close the slope file
slope_ds = None

NaturalFires['slope'] = slopes

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [13]:
from osgeo import gdal, osr
from pyproj import Proj, transform

# Open the DEM file
dem = gdal.Open('clipped.tif')  # replace 'clipped.tif' with your DEM file

# Define the target CRS (UTM zone 29T)
target_crs = osr.SpatialReference()
target_crs.ImportFromEPSG(32629)

# Reproject the DEM data
reprojected_dem = gdal.Warp('reprojected.tif', dem, dstSRS=target_crs)

# Calculate the slope
gdal.DEMProcessing('slope.tif', reprojected_dem, 'slope')

# Close the DEM file
dem = None
reprojected_dem = None

# Open the slope file
slope_ds = gdal.Open('slope.tif')

# Get the CRS of the raster data
dst_crs = Proj(slope_ds.GetProjection())

# Define the source CRS (WGS84)
src_crs = Proj(init='EPSG:4326')

slopes = []
for i in range(len(NaturalFires)):
    latitude = NaturalFires['latitude'][i]
    longitude = NaturalFires['longitude'][i]

    # Convert the coordinates to the raster data's CRS
    longitude_proj, latitude_proj = transform(src_crs, dst_crs, longitude, latitude)

    # Convert the projected coordinates to raster indices
    gt = slope_ds.GetGeoTransform()
    col = int((longitude_proj - gt[0]) / gt[1])
    row = int((latitude_proj - gt[3]) / gt[5])

    # Read the slope data
    slope_data = slope_ds.ReadAsArray()

    # Check if the indices are within the dimensions of the array
    if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
        slope_value = slope_data[row, col]
        print(slope_value)
        slopes.append(slope_value)
    else:
        print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")
        slopes.append(None)

# Close the slope file
slope_ds = None

NaturalFires['slope'] = slopes

/home/ori/.local/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


NameError: name 'NaturalFires' is not defined

In [ ]:
import rasterio

# Open the raster file
with rasterio.open('clipped.tif') as src:
    # Get the cell size
    cellsize_x, cellsize_y = src.res

print('Cell size in x direction:', cellsize_x)
print('Cell size in y direction:', cellsize_y)

In [16]:
def calculate_slope(dem, cellsize):
    def slope_filter(window):
        # Reshape the window to a 3x3 array
        matrix = window.reshape((3, 3))

        # Calculate the derivatives in the x and y directions
        dz_dx = (matrix[1, 2] - matrix[1, 0]) / (2 * cellsize)
        dz_dy = (matrix[2, 1] - matrix[0, 1]) / (2 * cellsize)

        # Calculate the slope
        slope = np.sqrt(dz_dx**2 + dz_dy**2)

        return slope

    return generic_filter(dem, slope_filter, size=3)

# Calculate slope
cellsize = 0.00833333333333337  # replace with your actual cell size
slope = calculate_slope(raster_data, cellsize)

In [14]:
def calculate_slope(dem):
    # Calculate the derivatives in the x and y directions
    dz_dx = (dem[1, 2] - dem[1, 0]) / 2
    dz_dy = (dem[2, 1] - dem[0, 1]) / 2

    # Calculate the slope
    slope = np.sqrt(dz_dx**2 + dz_dy**2)

    return slope

def calculate_slope_at_coordinate(dem, lon, lat, transform):
    # Convert the geographic coordinates to the raster's coordinate system
    col, row = ~transform * (lon, lat)

    # Make sure the row and column are integers
    col = int(col)
    row = int(row)

    # Check if the coordinate is within the bounds of the raster
    if row <= 0 or row >= dem.shape[0] - 1 or col <= 0 or col >= dem.shape[1] - 1:
        raise ValueError('The coordinate is on the edge of the raster or outside the raster')

    # Extract a 3x3 window around the cell
    window = dem[row-1:row+2, col-1:col+2]

    # Calculate the slope for the window
    slope = calculate_slope(window)

    return slope


# Get the raster's transform
with rasterio.open('clipped.tif') as src:
    transform = src.transform

# Calculate the slope at the specific geographic coordinate
lon = 37.283231  # replace with your actual longitude
lat = -8.5772292  # replace with your actual latitude
slope = calculate_slope_at_coordinate(raster_data, lon, lat, transform)
print(slope)

ValueError: The coordinate is on the edge of the raster or outside the raster

In [15]:
with rasterio.open('clipped.tif') as src:
    transform = src.transform
    bounds = src.bounds

# Check if the coordinate is within the bounds of the raster
lon = 37.283231  # replace with your actual longitude
lat = -8.5772292  # replace with your actual latitude
if not (bounds.left < lon < bounds.right and bounds.bottom < lat < bounds.top):
    raise ValueError('The coordinate is outside the raster')

# Calculate the slope at the specific geographic coordinate
slope = calculate_slope_at_coordinate(raster_data, lon, lat, transform)
print(slope)

ValueError: The coordinate is outside the raster

In [1]:
from osgeo import gdal, osr
from pyproj import Proj, transform

# Open the DEM file
dem = gdal.Open('md75_grd/md75_grd/w001001.adf')

# Define the target CRS (World Mercator)
target_crs = osr.SpatialReference()
target_crs.ImportFromEPSG(3395)

# Reproject the DEM data
reprojected_dem = gdal.Warp('reprojected.tif', dem, dstSRS=target_crs)

# Calculate the slope
gdal.DEMProcessing('slope.tif', reprojected_dem, 'slope')

# Close the DEM file
dem = None
reprojected_dem = None

# Open the slope file
slope_ds = gdal.Open('slope.tif')

# Get the CRS of the raster data
dst_crs = Proj(slope_ds.GetProjection())

# Define the source CRS (WGS84)
src_crs = Proj(init='EPSG:4326')

# Define the coordinate
latitude = 39.4  # replace with your latitude
longitude = -8.2  # replace with your longitude

# Convert the coordinate to the raster data's CRS
longitude_proj, latitude_proj = transform(src_crs, dst_crs, latitude, longitude)

# Convert the projected coordinate to raster indices
gt = slope_ds.GetGeoTransform()
col = int((longitude_proj - gt[0]) / gt[1])
row = int((latitude_proj - gt[3]) / gt[5])

# Read the slope data
slope_data = slope_ds.ReadAsArray()

# Check if the indices are within the dimensions of the array
if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
    slope_value = slope_data[row, col]
    print(slope_value)
else:
    print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")

# Close the slope file
slope_ds = None

ERROR 3: reprojected.tif: Free disk space available is 10578526208 bytes, whereas 104810947914 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.


ValueError: Received a NULL pointer.

In [12]:
import geopandas as gpd
from osgeo import gdal, osr
from pyproj import CRS, Transformer

# Load the shapefile of Portugal
portugal = gpd.read_file('ne_10m_admin_0_countries_prt/ne_10m_admin_0_countries_prt.shp')

# Get the bounding box of Portugal
west, south, east, north = portugal.total_bounds

# Convert the bounding box to the target CRS
transformer = Transformer.from_crs(portugal.crs, target_crs)
west_proj, south_proj = transformer.transform(south, west)
east_proj, north_proj = transformer.transform(north, east)

# Clip the DEM data to the extent of Portugal and reduce its resolution
clip_options = gdal.WarpOptions(outputBounds=[west_proj, south_proj, east_proj, north_proj], xRes=pixel_size, yRes=pixel_size, dstSRS=target_crs)
gdal.Warp('clipped.tif', dem, options=clip_options)

# Open the DEM file
dem = gdal.Open('md75_grd/md75_grd/w001001.adf')

# Define the target CRS
target_crs = osr.SpatialReference()
target_crs.ImportFromEPSG(32629)  # UTM zone 29T

# Define the pixel size
pixel_size = 0.01  # adjust as needed

# Open the clipped DEM file
clipped_dem = gdal.Open('clipped.tif')

# Calculate the slope
gdal.DEMProcessing('slope.tif', clipped_dem, 'slope')

# Close the DEM file and the clipped DEM
dem = None
clipped_dem = None

# Open the slope file
slope_ds = gdal.Open('slope.tif')

# Get the CRS of the raster data
dst_crs = dst_crs = CRS.from_string(slope_ds.GetProjection())

# Define the source CRS (WGS84)
src_crs = CRS.from_epsg(4326)

# Convert the bounding box to the target CRS
transformer = Transformer.from_crs(CRS.from_user_input(portugal.crs), target_crs)

# Define the coordinate
latitude = 37.283231  # replace with your latitude
longitude = -8.5772292  # replace with your longitude

# Convert the coordinate to the raster data's CRS
longitude_proj, latitude_proj = transformer.transform(longitude, latitude)

# Convert the projected coordinate to raster indices
gt = slope_ds.GetGeoTransform()
col = int((longitude_proj - gt[0]) / gt[1])
row = int((latitude_proj - gt[3]) / gt[5])

# Read the slope data
slope_data = slope_ds.ReadAsArray()

# Check if the indices are within the dimensions of the array
if 0 <= row < slope_data.shape[0] and 0 <= col < slope_data.shape[1]:
    slope_value = slope_data[row, col]
    print(slope_value)
else:
    print(f"The coordinates ({longitude}, {latitude}) are outside the extent of the raster.")

# Close the slope file
slope_ds = None

CRSError: Invalid CRS input: <osgeo.osr.SpatialReference; proxy of <Swig Object of type 'OSRSpatialReferenceShadow *' at 0x7fd157036cd0> >